# Parser 10.00 - Array Numeric Variables

This goal of this version of the parser it to teach it to parse expressions containing numeric array operands.

This is an even more limited goal than it might first appear.

For one thing, since we do not require variables to be declared, there will be no way for the parser to know the size of an array element or how many elements there are. Thus it will not be possible to produce the implicit expressions necessary to access a particular element of a classic [linear array](https://en.wikipedia.org/wiki/Array_data_structure), or to determine if an index value is actually outside the array.

Since we are not implementing statements of any sort, there will never be a looping statement that makes it possible to iterate over all array elements.

Nevertheless we can get a general idea of one way to go about parsing array notation. We are going to parse array operands of the form:

```Python
name[index]
```

where *name* is a numeric variable symbol and *index* is a numeric expression.

The left (**'\['**) and right (**'\]'**)bracket symbols delimiting *index* serve much the same function as parentheses. They guarantee that the enclosed expression is completely evaluated before it is related to *name*. Like parentheses they occur in pairs, it is an error for one to appear without the other, and left must come before right.

We are not going to go so far as some languages (\**cough*\* [C](https://en.wikipedia.org/wiki/C_(programming_language)) \**cough*\*) and permit the form:

```Python
[index]name
```

>This works in C because *name* is really a [pointer](https://www.tutorialspoint.com/cprogramming/c_pointers.htm), *index* is a number, numbers can be added to pointers, and addition is commutative. Here it would mean having to watch for a left bracket in both the *wantoperand* and *wantoperator* states. Not impossible, but quite a nuisance.

## Libraries

In [ ]:
import glob       # for searching directories
import math       # for 'log--()' functions

import re         # for regular expressions

## User output

In [ ]:
visSep = '-------------'             # visual separator

def UIwriteln(this):
    '''write a single line to output'''
    print( f'{this}\n' )
    
def UIwriteSep():
    '''write a visual separator'''
    UIwriteln( visSep )

def UIshow(tag, value):
    '''write a tagged value to output'''
    UIwriteln( f'{tag}: {value}' )

def UIerror(this):
    '''write an error message to output'''
    UIshow( 'Error', this )

# Tracing

In [ ]:
# flags: show trace of processing

showInteract = True          # default for interactive use
showBatch = False            # default for batch use

showTrace = None             # control flag

# Trace Output

def TOshow(mesg, text):
    '''write trace message to output if enabled'''
    if showTrace:
        UIshow( f'{mesg:15s}', text )
        
def TOstring(tag, this):
    
    if showTrace:
        TOshow( tag, ' '.join([str(e) for e in this]) )

# -----------------------
# Parse Tracing
# -----------------------

def PTshowexpr(this):

    TOshow( 'Parse', visSep )
    TOshow( 'Current Expr', this )

def PTshowparse(ok, res, opStk, typStk):

    if ok:
        TOstring( 'Current RPN', res )
        TOstring( 'Operator Stack', opStk )
        TOstring( 'Type Stack', typStk )

def PTshowtoken(this):

    if not this[0] == ' ':
        TOshow( "Found Token", this )

# -----------------------
# Evaluation Tracing
# -----------------------

def ETshowtoken(this):
    
    TOshow( 'Eval', visSep )
    TOshow( 'Current token', this )

def ETshoweval(stk):
    
    TOstring( 'Operand Stack', stk )


# Parser

In [ ]:
# operands accepted:
# - decimal and hexadecimal floating point literals
# - scalar and array numeric variables

# operators accepted:
# - unary negation, plus
# - binary addition, subtraction, multiplication, division
# - grouping parentheses
# - logical not, equality, inequality
# - assignment and shortcut assignment
# - prefix and postfix increment and decrement

# errors detected:
# - unrecognized input
# - out of range numeric input
# - malformed expression

# result tuple:
# - (True, [parse])
# - (False, None)

class Parser(object):
    
    VERSIONNUMBER = '10.00'
    
    _FLTMAX =  4294967295                                  # 2**32-1
    _FLTMIN = -4294967296                                  # -(2**32)

    _expMax = {
        'P' : math.log2(_FLTMAX),                          # max base 2 exponent
        'E' : math.log10(_FLTMAX)                          # max base 10 exponent
    }

    _typeChkLst = {                                         # type check control lists
         'n2n': ['number', 'number'],                       # - index is left-to-right
         'v2n': ['number', 'numsym'],
        'nn2n': ['number', 'number', 'number'],             # check is right-to-left
        'vn2n': ['number', 'numsym', 'number'],
        'vn2v': ['numsym', 'numsym', 'number']
    }
    
    def __init__(self):
        pass
        
    def doparse(self, this):

        def parseErr(mesg, pos):
            '''report parse error'''
            UIerror(mesg)
            UIwriteln(f'>>> {this}')
            if pos > 0:
                UIwriteln(f'{"^^near here".rjust(pos)}')
            return False

        # initialize

        expr = this                # save to new variable but retain original for error reports
        start = 15                 # tracked so we can report where in an expression an error occurred
        token = None               # anything successfully matched
        ok = wantoperand = True    # flags
        result = []                # rpn expression
        opStk = [ ('EOE', 1) ]     # operator stack
        typStk = []                # type stack
        

        def typeCheck(op, chk):
            '''type check operands'''            
            check = list(self._typeChkLst[chk])       # list() to avoid aliasing
            while len(check) > 1:
                want = check.pop()
                have, rpnPos, errPos = typStk.pop()
                
                # do we want a number ?
                
                if want == 'number':
                    
                    # convert symbols to numbers
                    
                    if have == 'numsym':
                        result.insert( rpnPos, 'U*' )
                        
                        
                # do we want a variable ?
                
                elif want == 'numsym':
                    
                    if have != 'numsym':
                        return parseErr( 'Numeric variable required', errPos )
                        
            # push result type, RPN operator position, original operator position
                    
            typStk.append( (check[0], len(result), errPos) )
            return True
 
        def addOperand(op, typ):
            '''add operand to RPN'''
            result.append( op )
            typStk.append( (typ, len(result), start) )
            
        def addOperator():
            '''add operator to RPN'''            
            op, _, chk = opStk.pop()
            result.append( op )
            return typeCheck( op, chk )
            
        def popGEop(prec):
            '''pop operators of equal or greater precedence'''
            ok = True
            while ok and prec <= opStk[-1][1]:
                ok = addOperator()
            return ok
 
        def pushLeft(op, prec, chk):
            '''push left associative operator on stack'''
            if not popGEop(prec):
                return False
            opStk.append( (op, prec, chk) )
            return True

        def popGop(prec):
            '''pop operators of greater precedence'''
            ok = True
            while ok and prec < opStk[-1][1]:
                ok = addOperator()
            return ok
 
        def pushRight(op, prec, chk):
            '''push right associative operator on stack'''
            if not popGop(prec):
                return False
            opStk.append( (op, prec, chk) )
            return True

        def popUntil(op, prec):
            '''clear and check operator stack'''
            if not popGEop(prec):                       # type check failure ?
                return False

            topop = opStk.pop()[0]                      # found match ?
            if op == topop:
                return True
            
            elif op == '(':
                err = 'right parenthesis'
            elif op == '[':
                err = 'right bracket'
            elif topop == '(':
                err = 'left parenthesis'
            elif topop == '[':
                err = 'left bracket'
            else:
                err = 'EOE'
            
            return parseErr( f'Unmatched {err}', start )
   
        # operator dictionaries initialization
        
        _postOps = '[+]{2}|[-]{2}'
        
        _postOp = {
            '++': (90, pushLeft, 'v2n'),
            '--': (90, pushLeft, 'v2n')
        }

        _unOps = '!|[+]{1,2}|[-]{1,2}'

        _unOp =  {
            '-':  (80, pushRight, 'n2n'),
            '+':  (80, pushRight, 'n2n'),
            '!':  (80, pushRight, 'n2n'),
            '++': (80, pushRight, 'v2n'),
            '--': (80, pushRight, 'v2n')
        }

        _binOps = '[-+*/]=?|[=!]?=|\['

        _binOp = {
            '[':  (90, pushLeft,  'vn2v'),
            '*':  (70, pushLeft,  'nn2n'),
            '/':  (70, pushLeft,  'nn2n'),
            '+':  (60, pushLeft,  'nn2n'),
            '-':  (60, pushLeft,  'nn2n'),
            '==': (50, pushLeft,  'nn2n'),
            '!=': (50, pushLeft,  'nn2n'),
            '=':  (10, pushRight, 'vn2n'),
            '*=': (10, pushRight, 'vn2n'),
            '/=': (10, pushRight, 'vn2n'),
            '+=': (10, pushRight, 'vn2n'),
            '-=': (10, pushRight, 'vn2n')
        }
      
        def convertFloat(fplit, base, capgrp):
            '''convert floating point literal to internal form'''
            
            def rangeErr():
                return parseErr(f'\'{fplit}\' is out of range', start)
            
            # collect the features of interest
                   
            p = re.search(capgrp, fplit.upper())
                            
            lint, lfrc, expbas, expsgn, lexp = p.group(1,2,4,5,6)
            
            # convert integer portion (if any)
 
            uint = 0
            if lint:
                p = re.search('[1-9A-F][0-9A-F]*', lint )
                if p != None:
                    for ch in p.group():
                        digval = '0123456789ABCDEF'.find(ch)
                        if uint <= (self._FLTMAX - digval)/base:
                            uint =  uint * base + digval
                        else:
                            return rangeErr()
                    
            # convert fractional portion (if any)
                    
            ufrc = 0
            if lfrc:
                fbase = 1
                for ch in lfrc:
                    digval = '0123456789ABCDEF'.find(ch)
                    fbase *= base
                    ufrc += digval/fbase
        
            if uint == self._FLTMAX and ufrc != 0:
                return rangeErr()
            
            # value so far
            
            uflt = uint + ufrc
 
            # convert exponent portion (if any)
            
            uexp = 0
            if lexp:
                for ch in lexp:
                    digval = '0123456789'.find(ch)
                    if uexp <= (self._expMax[expbas] - digval)/10:
                        uexp =  uexp * 10 + digval
                    else:
                        return rangeErr()
                    
            # adjust value by exponent (if any)
             
            if uexp:
                power = (2 if expbas == 'P' else 10) ** uexp
                if expsgn == '-':
                    uflt /= power
                elif uflt <= self._FLTMAX/power:
                    uflt *= power
                else:
                    return rangeErr()
                    
            addOperand( uflt, 'numlit' )
            return True

        def startsWith(regex):
            '''test if expression starts with given regular expression'''
            nonlocal expr, start, token

            p = re.match(regex, expr)
            if p == None:
                return False
            else:
                token = p.group()              # what we matched
                start += len(token)            # update to next match position in original string
                expr = expr[len(token):]       # "chop off" what we matched
                PTshowtoken(token)             # trace
                return True

        # top level main loop
              
        while ok and len(expr):

            _ = startsWith('[ ]+')                             # skip leading whitespace

            PTshowexpr(expr)                                   # trace

            # look for operand

            if wantoperand:

                if startsWith('[(]'):
                    '''left parenthesis ?'''
                    opStk.append( ('(', 2) )

                elif startsWith(_unOps):
                    '''right unary?'''
                    prec, assoc, check = _unOp[token]
                    assoc( 'U' + token, prec, check )

                else:

                    wantoperand = False                         # flip
                    
                    if startsWith('0[xX]([0-9a-fA-F]+([.][0-9a-fA-F]*)?|[.][0-9a-fA-F]+)([pP][-+]?[0-9]+)?'):
                        '''unsigned hexadecimal literal ?'''
                        ok = convertFloat(token, 16, '0X([0-9A-F]*)[.]?([0-9A-F]*)(([P])([-+])?([0-9]+))?')

                    elif startsWith('([0-9]+([.][0-9]*)?|[.][0-9]+)([eE][-+]?[0-9]+)?'):
                        '''unsigned decimal literal?'''
                        ok = convertFloat(token, 10, '([0-9]*)[.]?([0-9]*)(([E])([-+])?([0-9]+))?')
                        
                    elif startsWith('[a-zA-Z][_a-zA-Z0-9]*'):
                        '''numeric scalar variable?'''
                        addOperand( token.upper(), 'numsym')
                        
                    else:
                        '''malformed'''
                        ok = parseErr('Expecting operand', start)

            # look for operator

            else:

                if startsWith('\)|\]'):
                    ok = popUntil( '(' if token == ')' else '[', 4 )
                    
                elif startsWith(_postOps):
                    '''postfix inc/dec?'''
                    prec, assoc, check = _postOp[token]
                    ok = assoc( 'P' + token, prec, check )

                else:

                    wantoperand = True                                # flip
                    
                    if startsWith(_binOps):
                        '''binary operator?'''
                        prec, assoc, check = _binOp[token]
                        ok = assoc( 'B' + token, prec, check )
                        if ok and token == '[':
                            opStk.append( (token, 2) )

                    else:
                        '''malformed'''
                        ok = parseErr('Expecting operator', start)

            PTshowparse(ok, result, opStk, typStk )               # trace
 
        if ok and wantoperand:
            ok = parseErr('Unexpected end of expression', start ) # must be in 'wantoperator' state   

        if ok:
            ok = popUntil( 'EOE', 3 )                             # clear operator stack
            
        if ok:
            ok = typeCheck('LoneSymbol', 'n2n')                   # make sure final result is a number

        return (ok, result if ok else None)                       # done
       

### How it works

Array brackets are so similar to grouping parentheses that we can treat them almost exactly the same way. We can use the same precedence values and the same method of directly pushing the left one on the operator stack and having the right clear it off later.

A slight difference is that while a left parenthesis is legal only in the *wantoperand* state and does not change it, a left bracket is legal only in the *wantoperator* state and *does* change it. A left bracket follows an operand and we want an operand to follow it. In this sense it naturally *is* an operator.

Once a right bracket clears its matching left bracket off the operator stack, *name* and *index* are in the Reverse Polish and the brackets themselves are gone. But we still need to connect them together.

For a true linear array this would be fairly straightforward. These commonly exist as blocks of memory large enough to hold all their elements, one after another. Accessing a particular element involves multiplying its index by the size of one element and adding the result to the start address of the block.

We can’t do that directly in Python because its arrays are not linear but [associative](https://en.wikipedia.org/wiki/Associative_array). These do not even intrinsically *have* numeric indices (though we can bolt them on later using the [*enumerate()*](https://realpython.com/python-enumerate/) method if we like).

But we can achieve an effect that appears the same as a linear array. If we treat *index* as a string instead of a number and concatenate it to *name* at evaluation time, we can create a unique symbol name to use with our existing symbol table. Since we will use type checking to guarantee that *name* is a variable and *index* is a number, from the outside this looks just like accessing a linear array.

>Note that all index expressions which result in the same value will create the same *name* + *index* entry in the symbol table.

We will add the left bracket **'\['** to our collection of binary operators in *_binOps*. It has the same precedence and associativity as the post-increment and -decrement operators, but a different type check. We add all these to *_binOp{}*.

*typeChkLst{}* gets a new entry, *'vn2v'*, to handle the new check. The result type is *numsym*, a variable. Values can be assigned (written to it) and de-referenced (read from it).

We search for a left bracket the same as for any other binary operator. When found its operator form *B[* is placed on the operator stack. We immediately "cover it up" with a low precedence bare *\[* "operator". This protects the high precedence *B[* from being popped off the operator stack until after a matching right bracket has been found. Once it has been exposed it will be popped off by either the next operator found or the end of the expression. Either way, it will appear immediately after the index expression. Which is what we want.

In the *wantoperator* state we start by looking for a right bracket or parenthesis. We use the regular expression

```Python
'\)|\]'
```

instead of the character class trick we have been using up until now, which would look like:

```Python
'[)\]]'
```

not because it doesn't work, but because Python thinks they look suspiciously nested and issues a warning. (even though the first **']'** is escaped).

>Even though the first **']'** is expliticly [escaped](https://learnbyexample.github.io/py_regular_expressions/escaping-metacharacters.html). Which is annoying, but metacharacters. What can we do?

The *popUntil()* function that clears the operator stack down to a specific token now has to identify bracket as well as grouping parenthesis mis-matches.

#### Multi-dimensional Arrays

Perhaps surprisingly, we don’t have to do anything more to successfully parse multi-dimensional arrays with any number of indices. For example, a two-dimensional array reference looks like:

```Python
name[index1][index2]
```

Recall that a right bracket is encountered in the *wantoperator* state but does not change that state. A left bracket *is* an operator, and there is no difficulty in allowing one to directly follow a right bracket.

A *B\[* operator leaves a type of “numsym” on the type stack. When the right bracket delimiting the second – or third or fourth or whatever – dimension adds the next *B[* to the Reverse Polish, its type check will find that result in exactly the place it needs to be.

And so we have multi-dimensional arrays with no additional effort.

# Evaluator

In [ ]:
# operators handled:
# - unary negation, plus
# - binary addition, subtraction, multiplication, division
# - logical negation, equality and inequality
# - variable name de-reference
# - variable assignment and shortcut assignment
# - prefix and postfix increment and decrement
# - numeric array assignment and de-reference

# errors detected:
# - out of range
# - division by zero

# return tuple:
# - (True, result)
# - (False, None)

class Evaluator(Parser):
    
    def __init__(self):
        self._symTable = dict()
 
    def doeval(self, rpn):
 
        def inRange(ok, val):
            '''range check test result'''
            if ok:
                stk.append( val )
            else:
                UIerror( 'Evaluation result out of range' )
            return ok
        
        def pushOperand(val):
            '''push operand on stack'''
            stk.append( val )
            return True
        
        def binAryNdx(ndx, nam):
            '''create array index'''
            ndx = round(ndx)
            return inRange( 0 <= ndx <= self._FLTMAX, f'{nam}_{ndx}' )
                     
        def binVal(val, var):
            '''assign value to variable'''
            self._symTable[var] = val
            return pushOperand(val)
        
        def unVal(var):
            '''variable name value'''
            return pushOperand(self._symTable[var] if var in self._symTable else 0)

#        def unPlu(arg):
#           '''unary plus'''
#           return pushOperand( arg )

        def unNeg(arg):
            '''unary negation'''
            return inRange( arg != self._FLTMIN, -arg )
        
        def unNot(arg):
            '''logical not'''
            return pushOperand( not arg )

        def binAdd(rgt, lft):
            '''binary addition'''
            if lft >= 0:
                return inRange( rgt <= self._FLTMAX - lft, lft+rgt )       
            else:
                return inRange( rgt >= self._FLTMIN - lft, lft+rgt )
 
        def binSub(rgt, lft):
            '''binary subtraction'''
            if lft >= 0:
                return inRange( lft - self._FLTMAX <= rgt, lft-rgt )
            elif rgt >= 0:
                return inRange( lft - self._FLTMIN >= rgt, lft-rgt )
            else:
                return inRange( lft <= self._FLTMAX + rgt, lft-rgt )
 
        def binMul(rgt, lft):
            '''binary multiplication'''
            if abs(lft) <= 1 or abs(rgt) <= 1:
                return pushOperand( lft * rgt )

            elif lft > 0:
                if rgt > 0:
                    return inRange( rgt <= self._FLTMAX / lft, lft * rgt )
                else:
                    return inRange( rgt >= self._FLTMIN / lft, lft * rgt )

            elif rgt > 0:
                return inRange( rgt <= self._FLTMIN / lft, lft * rgt )
  
            else:
                return inRange( rgt >= self._FLTMAX / lft, lft * rgt )
 
        def binDiv(rgt, lft):
            '''binary division'''
            if abs(rgt) >= 1:
                return pushOperand( lft/rgt )
            
            elif rgt > 0:
                if lft > 0 :
                    return inRange( lft <= self._FLTMAX * rgt, lft / rgt )
                else:
                    return inRange( lft >= self._FLTMIN * rgt, lft / rgt )
            
            elif rgt < 0:
                if lft > 0:
                    return inRange( lft <= self._FLTMIN * rgt, lft / rgt )
                else:
                    return inRange( lft >= self._FLTMAX * rgt, lft / rgt )

            else:
                UIerror( 'Division by zero' )
                return False
            
        def binEqu(rgt, lft):
            '''logical equality'''
            return pushOperand( lft == rgt )
        
        def binNeq(rgt, lft):
            '''logical inequality'''
            return pushOperand( lft != rgt )
        
        def binShortVal(val, var, op):
            '''shortcut assignment'''
            _ = unVal(var)                           # put the value of the variable on the stack
            if op(val, stk.pop()):                   # perform the arithmetic
                return binVal(stk.pop(), var)        # if successful,  also assign result to variable
            return False
        
        def binAddVal(val, var):
            return binShortVal(val, var, binAdd)
        
        def binSubVal(val, var):
            return binShortVal(val, var, binSub)
        
        def binMulVal(val, var):
            return binShortVal(val, var, binMul)
        
        def binDivVal(val, var):
            return binShortVal(val, var, binDiv)
        
        def unPfxInc(var):
            return binShortVal(1, var, binAdd)
        
        def unPfxDec(var):
            return binShortVal(1, var, binSub)
        
        def unPostFix(val, var):
            '''postfix inc/dec'''
            _ = unVal(var)                             # push current value of variable on stack
            ok = binShortVal(val, var, binAdd)         # update value of variable
            stk.pop()                                  # remove updated value from stack (if error, removes first push)
            return ok
                
        def unPstInc(var):
            return unPostFix(1, var)
            
        def unPstDec(var):
            return unPostFix(-1, var)
        
        # initialize
                                
        unDispatch = {
            'U*': unVal,
            'U-': unNeg,
            'U+': pushOperand,
            'U!': unNot,
            'U++': unPfxInc,
            'U--': unPfxDec,
            'P++': unPstInc,
            'P--': unPstDec
        }
        
        binDispatch = {
            'B+': binAdd, 'B-': binSub,
            'B+=': binAddVal, 'B-=': binSubVal,
            'B*': binMul, 'B/': binDiv,
            'B*=': binMulVal, 'B/=': binDivVal,
            'B==': binEqu, 'B!=': binNeq,
            'B=': binVal, 'B[': binAryNdx
        }
  
        stk = []
        ok = True

        # main loop
        
        TOstring('Input', rpn)
        
        for v in rpn:

            ETshowtoken(v)
            
            if v in binDispatch:
                ok = binDispatch[v](stk.pop(), stk.pop())
                
            elif v in unDispatch:
                ok = unDispatch[v](stk.pop())
                
            else:
                stk.append( v )

            if not ok:
                return ( False, None )

            ETshoweval( stk )

        return ( True, stk.pop() )


### How it works

*B\[* is given an entry in *_binDispatch{}* that points to *binAryNdx()*. This function creates an index into the symbol dictionary by concatentating *name* and *index*, then leaves the result on the operand stack. It does not reference the symbol dictionary itself but instead leaves that to other operators. This makes it possible to use *B[* repeatedly to build up a "multi-dimensional" array index.

*binAryNdx()* range checks *index* mainly to disallow negative indices. Python itself would happily accept them, but this restriction makes it a bit easier to maintain the illusion of a zero-based linear array.

>This illusion can be quickly destroyed by use of a non-integer index, of course, so we use Pythno's *round()* function to guarantee we have one.

That the upper limit is the same as the maximum allowed value in some ways makes that check redundant. Any attempt to create a larger value will have already failed before reaching this point. But it can’t hurt, and it might not always be redundant if an ill-advised change elsewhere makes it possible for an illegal value to slip through.

Although overall numeric array operands do not involve much additional code, quite complicated expressions can now be parsed and evaluated. A few examples:

```Python
a[1] = 2
b[6] = a[1]
c[m=5] = b[3+3]
d[1][17] = c[5]
e[c[m]][a[1]] = d[b[6]-1][m+c[5]*6]
```

Not a bad return for a fairly small investment.

## Running the parser

In [ ]:
passCnt = failCnt = 0                       # most useful for test input files, but never any harm

myParser = myEvaluator = None               # where we keep instances of our classes

def startUp(flag):
    '''begin execution'''
    global passCnt, failCnt, showTrace
    global myParser, myEvaluator
    if not myParser:
        myParser = Parser()
    if not myEvaluator:
        myEvaluator = Evaluator()
    UIshow( 'Parser', myParser.VERSIONNUMBER )
    passCnt = failCnt = 0
    showTrace = flag
    
def shutDown():
    '''terminate execution'''
    UIwriteSep()
    UIshow( 'Pass', passCnt )
    UIshow( 'Fail', failCnt )
    
# run parser
        
def parseOne(this):
    '''parse/evaluate one expression'''
    global passCnt, failCnt
    UIwriteSep()
    neg = this[0] == '@'
    if neg:
        this = this[1:]
    UIshow( 'Input', this )
    ok, res = myParser.doparse( this )
    if ok:
        UIshow( 'Final Parse', res )
        ok, res = myEvaluator.doeval( res )
        if ok:
            UIshow( 'Final Eval', res )
        if neg:
            ok = not ok
    if ok:
        passCnt += 1
    else:
        failCnt += 1

## Interactive use

In [11]:
def parse():
    
    startUp(showInteract)
    while True:
        inp = input( 'Expression: ' )
        UIwriteln( '' )                      # looks better with a blank line here
        if inp.upper()[0] == 'Q':
            break
        elif inp.strip():
            parseOne( inp )
    shutDown()

## Batch processing

In [ ]:
testDir = '..\\ParserTest\\'            # directory holding test input files (empty string if same as notebook directory)

# convert current version number to match test file numbers
# - done this way so we can update only the version number and everything still works

def currNum():
    
    head = myParser.VERSIONNUMBER[:len(myParser.VERSIONNUMBER)-3]
    tail = myParser.VERSIONNUMBER[-2:]
    return f'{head:0>2}{tail}'

# make full path name to test file

def makePath(typ, num):
    return f'{testDir}{typ}{num}.txt'

# run one test

def runTest(this):
    
    UIwriteln(f'Parser {myParser.VERSIONNUMBER} vs {this[-12:-4]}')
    
    myEvaluator._symTable.clear()
    
    with open(this) as f:
        data = f.readlines()
    for line in data:
        test = line.strip()
        if test and test[0] != '#':         # skip blank and comment lines
            parseOne(test)
    
# run a test of current or specified version which should succeed
    
def good(num='curr'):
  
    startUp(showBatch)
    runTest(makePath('pass', currNum() if num == 'curr' else num))
    shutDown()
    
# run a test of current or specified version which should fail

def bad(num='curr'):
    
    startUp(showBatch)
    runTest(makePath('fail', currNum() if num == 'curr' else num))
    shutDown()
    
# run regression test against current and all previous test files

def regress():
            
    UIwriteln('PASS tests')
    
    startUp(showBatch)                       # must create objects before we can access variables inside them 
    currFn = makePath('pass', currNum())
    failed = []
    fnlist = glob.glob(f'{testDir}pass????.txt')
    for fn in fnlist:
        if fn <= currFn:
            atstart = failCnt
            runTest(fn)
            if atstart < failCnt:
                failed.append(fn)               
    shutDown()
    
    UIwriteln('FAIL tests')
    
    startUp(showBatch)
    currFn = makePath('fail',currNum())
    passed = []
    fnlist = glob.glob(f'{testDir}fail????.txt')
    for fn in fnlist:
        if fn <= currFn:
            atstart = passCnt
            runTest(fn)
            if atstart < passCnt:
                passed.append(fn)                
    shutDown()
    
    if not len(failed):
        UIwriteln('All pass tests succeded')
    else:
        UIwriteln('Pass tests which failed')
        for fn in failed:
            UIwriteln(f'  {fn}')
            
    if not len(passed):
        UIwriteln('All fail tests succeded')
    else:
        UIwriteln('Fail tests which passed')
        for fn in passed:
            UIwriteln(f'   {fn}')
              

# Testing the parser

In [ ]:
parse()       # interactive, one expression at a time

In [ ]:
good()        # run current parser against its own pass test. Use good('1234') to run against specific pass test.

In [ ]:
bad()         # run current parser against its own fail test. Use bad('5678') to run against specific fail test.

In [ ]:
regress()     # run parser against all previous tests